W207: Final Project
===========

Our team (Danish Iqbal, KC Tobin, Jason Vantomme) is working through the Kaggle "House Prices: Advanced Regression Techniques" competition @ https://www.kaggle.com/c/house-prices-advanced-regression-techniques.

Our first step is to scrub the data and generate effective features from what is provided.  We have divided this process into three parts, one for each team member.

In [1]:
# ENVIRONMENT SETUP

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import datetime as dt
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures as PF, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model as lm
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import norm
from scipy import stats

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.precision',6)
pd.set_option('display.max_columns',None) 
pd.set_option('display.max_rows',None) 
pd.set_option('display.max_colwidth',1000)

In [2]:
# make sure you have the latest version of pandas (>=0.19.0) otherwise the 'category' type will throw an error

colTypesDict = {"MSSubClass":'category',"MSZoning":'category',"Street":'category',"Alley":'category',
                "LotShape":'category',"LandContour":'category',"Utilities":'category',
                "LotConfig":'category',"LandSlope":'category',"Neighborhood":'category',"Condition1":'category',
                "Condition2":'category',"BldgType":'category',"HouseStyle":'category',"RoofStyle":'category',
                "RoofMatl":'category',"Exterior1st":'category',"Exterior2nd":'category',"MasVnrType":'category',
                "ExterQual":'category',"ExterCond":'category',"Foundation":'category',
                "BsmtQual":'category',"BsmtCond":'category',"BsmtExposure":'category',"BsmtFinType1":'category',
                "BsmtFinType2":'category',"Heating":'category',"HeatingQC":'category',"CentralAir":'category',
                "Electrical":'category',"KitchenQual":'category',"Functional":'category',"FireplaceQu":'category',
                "GarageType":'category',"GarageFinish":'category',
                "GarageQual":'category',"GarageCond":'category',"PavedDrive":'category',
                "PoolQC":'category',"Fence":'category',"MiscFeature":'category',
                "SaleType":'category',"SaleCondition":'category'}

# LOAD THE PROVIDED TRAINING DATA
train = pd.read_csv('data/train.csv',delimiter=',',header=0,dtype=colTypesDict,na_values=["NA"])
test = pd.read_csv('data/test.csv',delimiter=',',header=0)

In [3]:
print train.shape
print test.shape

print train.describe(include='all')

(1460, 81)
(1459, 80)
                 Id MSSubClass MSZoning  LotFrontage        LotArea Street  \
count   1460.000000       1460     1460  1201.000000    1460.000000   1460   
unique          NaN         15        5          NaN            NaN      2   
top             NaN         20       RL          NaN            NaN   Pave   
freq            NaN        536     1151          NaN            NaN   1454   
mean     730.500000        NaN      NaN    70.049958   10516.828082    NaN   
std      421.610009        NaN      NaN    24.284752    9981.264932    NaN   
min        1.000000        NaN      NaN    21.000000    1300.000000    NaN   
25%      365.750000        NaN      NaN    59.000000    7553.500000    NaN   
50%      730.500000        NaN      NaN    69.000000    9478.500000    NaN   
75%     1095.250000        NaN      NaN    80.000000   11601.500000    NaN   
max     1460.000000        NaN      NaN   313.000000  215245.000000    NaN   

       Alley LotShape LandContour Utiliti

In [4]:
## SET 1 (Danish)

##

train['MasVnrArea'].fillna(0, inplace=True)
train['MasVnrArea2'] = train['MasVnrArea'].astype(int) 

train['MasVnrArea2_0'] = train['MasVnrArea2']==0
train['MasVnrArea2_50'] = (train['MasVnrArea2']>0) & (train['MasVnrArea2']<50)
train['MasVnrArea2_100'] = (train['MasVnrArea2']>50) & (train['MasVnrArea2']<100)
train['MasVnrArea2_150'] = (train['MasVnrArea2']>100) & (train['MasVnrArea2']<150)
train['MasVnrArea2_200'] = (train['MasVnrArea2']>150) & (train['MasVnrArea2']<200)
train['MasVnrArea2_250'] = (train['MasVnrArea2']>200) & (train['MasVnrArea2']<250)
train['MasVnrArea2_300'] = (train['MasVnrArea2']>250) & (train['MasVnrArea2']<300)
train['MasVnrArea2_350'] = (train['MasVnrArea2']>300) & (train['MasVnrArea2']<350)
train['MasVnrArea2_400'] = (train['MasVnrArea2']>350) & (train['MasVnrArea2']<400)
train['MasVnrArea2_other'] = (train['MasVnrArea2']>400)

#--

train['MSSubClass_60'] = train['MSSubClass']==60
train['MSSubClass_50'] = train['MSSubClass']==50
train['MSSubClass_120'] = train['MSSubClass']==120
train['MSSubClass_80'] = train['MSSubClass']==80
train['MSSubClass_160'] = train['MSSubClass']==160
train['MSSubClass_90'] = train['MSSubClass']==90
train['MSSubClass_30'] = train['MSSubClass']==30
train['MSSubClass_70'] = train['MSSubClass']==70
train['MSSubClass_other'] = (train['MSSubClass']==85) | (train['MSSubClass']==190) | (train['MSSubClass']==75)  | (train['MSSubClass']==180) | (train['MSSubClass']==45) | (train['MSSubClass']==40)

train['LotFrontage_none'] = train['LotFrontage']==0
train['LotFrontage_50'] = (train['LotFrontage']>0) & (train['LotFrontage']<50)
train['LotFrontage_60'] = (train['LotFrontage']>50) & (train['LotFrontage']<60)
train['LotFrontage_70'] = (train['LotFrontage']>60) & (train['LotFrontage']<70)
train['LotFrontage_80'] = (train['LotFrontage']>70) & (train['LotFrontage']<80)
train['LotFrontage_90'] = (train['LotFrontage']>80) & (train['LotFrontage']<90)
train['LotFrontage_other'] = train['LotFrontage']>90

train['LotArea_5'] = train['LotArea']<5000
train['LotArea_6'] = (train['LotArea']>5000) & (train['LotArea']<6000)
train['LotArea_7'] = (train['LotArea']>6000) & (train['LotArea']<7000)
train['LotArea_8'] = (train['LotArea']>7000) & (train['LotArea']<8000)
train['LotArea_9'] = (train['LotArea']>8000) & (train['LotArea']<9000)
train['LotArea_10'] = (train['LotArea']>9000) & (train['LotArea']<10000)
train['LotArea_11'] = (train['LotArea']>10000) & (train['LotArea']<11000)
train['LotArea_12'] = (train['LotArea']>11000) & (train['LotArea']<12000)
train['LotArea_13'] = (train['LotArea']>12000) & (train['LotArea']<13000)
train['LotArea_14'] = (train['LotArea']>13000) & (train['LotArea']<14000)
train['LotArea_other'] = (train['LotArea']>14000) 

train['YearBuilt_30'] = train['YearBuilt']<1930
train['YearBuilt_40'] = (train['YearBuilt']>1930) & (train['YearBuilt']<1940)
train['YearBuilt_50'] = (train['YearBuilt']>1940) & (train['YearBuilt']<1950)
train['YearBuilt_60'] = (train['YearBuilt']>1950) & (train['YearBuilt']<1960)
train['YearBuilt_70'] = (train['YearBuilt']>1960) & (train['YearBuilt']<1970)
train['YearBuilt_80'] = (train['YearBuilt']>1970) & (train['YearBuilt']<1980)
train['YearBuilt_90'] = (train['YearBuilt']>1980) & (train['YearBuilt']<1990)
train['YearBuilt_00'] = (train['YearBuilt']>1990) & (train['YearBuilt']<2005)

train['YearRemodAdd_50'] = (train['YearRemodAdd']>1950) & (train['YearRemodAdd']<1960)
train['YearRemodAdd_60'] = (train['YearRemodAdd']>1960) & (train['YearRemodAdd']<1970)
train['YearRemodAdd_70'] = (train['YearRemodAdd']>1970) & (train['YearRemodAdd']<1980)
train['YearRemodAdd_80'] = (train['YearRemodAdd']>1980) & (train['YearRemodAdd']<1990)
train['YearRemodAdd_90'] = (train['YearRemodAdd']>1990) & (train['YearRemodAdd']<2000)
train['YearRemodAdd_00'] = (train['YearRemodAdd']>2000)

#--
print train.shape
for col in train.columns:
    print col

(1460, 133)
Id
MSSubClass
MSZoning
LotFrontage
LotArea
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Heating
HeatingQC
CentralAir
Electrical
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
PoolQC
Fence
MiscFeature
MiscVal
MoSold
YrSold
SaleType
SaleCondition
SalePrice
MasVnrArea2
MasVnrArea2_0
MasVnrArea2_50
MasVnrArea2_100
MasVnrArea2_150
MasVnrArea2_200
MasVnrArea2_250
MasVnrArea2_300
MasVnrArea2_

In [5]:
## SET 2 (KC)

#handle NAs with train; categoricals
train["BsmtFinType2"] = train["BsmtFinType2"].fillna(method="ffill")
train["BsmtExposure"] = train["BsmtExposure"].fillna(method="ffill")

#handle NAs with test; ints
test["BsmtFinSF1"] = train["BsmtFinSF1"].fillna(0)
test["BsmtFinSF2"] = train["BsmtFinSF2"].fillna(0)
test["TotalBsmtSF"] = train["TotalBsmtSF"].fillna(0)
test["BsmtUnfSF"] = train["BsmtUnfSF"].fillna(0)
test["BsmtHalfBath"] = train["BsmtHalfBath"].fillna(0)
test["BsmtFullBath"] = train["BsmtFullBath"].fillna(0)

#handle NAs with test; categoricals
test["KitchenQual"] = test["KitchenQual"].fillna(method="ffill")
test["BsmtCond"] = test["BsmtCond"].fillna(method="ffill")
test["BsmtQual"] = test["BsmtQual"].fillna(method="ffill")
test["BsmtExposure"] = test["BsmtExposure"].fillna(method="ffill")

#create other rooms columns
train["OtherRmsAbvGr"] = train["TotRmsAbvGrd"] - (train["BedroomAbvGr"]+train["KitchenAbvGr"])
test["OtherRmsAbvGr"] = test["TotRmsAbvGrd"] - (test["BedroomAbvGr"]+test["KitchenAbvGr"])

#create binary columns
train["HasBsmt"] = train["TotalBsmtSF"] > 0
train["Has2ndFlr"] = train["2ndFlrSF"] > 0
test["HasBsmt"] = test["TotalBsmtSF"] > 0
test["Has2ndFlr"] = test["2ndFlrSF"] > 0

#drop total columns
train = train.drop(["GrLivArea", "TotalBsmtSF", "TotRmsAbvGrd"], axis = 1)
test = test.drop(["GrLivArea", "TotalBsmtSF", "TotRmsAbvGrd"], axis = 1)

#log transforms of area
train["BsmtFinSF1"] = np.log(train["BsmtFinSF1"]+1)
train["BsmtFinSF2"] = np.log(train["BsmtFinSF2"]+1)
train["BsmtUnfSF"] = np.log(train["BsmtUnfSF"]+1)
train["1stFlrSF"] = np.log(train["1stFlrSF"]+1)
train["2ndFlrSF"] = np.log(train["2ndFlrSF"]+1)
train["LowQualFinSF"] = np.log(train["LowQualFinSF"]+1)

test["BsmtFinSF1"] = np.log(test["BsmtFinSF1"]+1)
test["BsmtFinSF2"] = np.log(test["BsmtFinSF2"]+1)
test["BsmtUnfSF"] = np.log(test["BsmtUnfSF"]+1)
test["1stFlrSF"] = np.log(test["1stFlrSF"]+1)
test["2ndFlrSF"] = np.log(test["2ndFlrSF"]+1)
test["LowQualFinSF"] = np.log(test["LowQualFinSF"]+1)

print train.shape
for col in train.columns:
    print col

(1460, 133)
Id
MSSubClass
MSZoning
LotFrontage
LotArea
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
Heating
HeatingQC
CentralAir
Electrical
1stFlrSF
2ndFlrSF
LowQualFinSF
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
PoolQC
Fence
MiscFeature
MiscVal
MoSold
YrSold
SaleType
SaleCondition
SalePrice
MasVnrArea2
MasVnrArea2_0
MasVnrArea2_50
MasVnrArea2_100
MasVnrArea2_150
MasVnrArea2_200
MasVnrArea2_250
MasVnrArea2_300
MasVnrArea2_350
MasVnrArea2_400
MasVnrArea2_oth

In [6]:
## SET 3 (Jason)

##
##  PORCH/DECK COLLAPSE/REMOVALS INTO ONE SQ FT MEASUREMENT (SUM OF ALL)
##
porches = train.loc[:,["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]]
reduced_porches = porches.sum(axis=1)
train["TotalPorchSqFt"] = reduced_porches
#
porches = test.loc[:,["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]]
reduced_porches = porches.sum(axis=1)
test["TotalPorchSqFt"] = reduced_porches

## ----------------------------
##  POOL MODIFICATIONS
## ----------------------------
train["HasPool"] = train["PoolArea"] > 0
train = train.drop( ["PoolArea"], axis=1 )
#
test["HasPool"] = test["PoolArea"] > 0
test = test.drop( ["PoolArea"], axis=1 )

## ----------------------------
##  FIREPLACE COLLAPSE/REMOVALS
## ----------------------------
manyFireplaces = 2
train["HasManyFireplaces"] = train["Fireplaces"] > manyFireplaces
test["HasManyFireplaces"] = test["Fireplaces"] > manyFireplaces

## ----------------------------
##  GARAGE COLLAPSE/REMOVALS
## ----------------------------
largeGarageSize = 800
train["HasLargeGarage"] = train["GarageArea"] > largeGarageSize
train = train.drop( ["GarageArea"], axis=1 )
#
test["HasLargeGarage"] = test["GarageArea"] > largeGarageSize
test = test.drop( ["GarageArea"], axis=1 )

print train["GarageYrBlt"].dtypes

print train.shape
for col in train.columns:
    print col
    
    

float64
(1460, 135)
Id
MSSubClass
MSZoning
LotFrontage
LotArea
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
MasVnrArea
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinSF1
BsmtFinType2
BsmtFinSF2
BsmtUnfSF
Heating
HeatingQC
CentralAir
Electrical
1stFlrSF
2ndFlrSF
LowQualFinSF
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageQual
GarageCond
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolQC
Fence
MiscFeature
MiscVal
MoSold
YrSold
SaleType
SaleCondition
SalePrice
MasVnrArea2
MasVnrArea2_0
MasVnrArea2_50
MasVnrArea2_100
MasVnrArea2_150
MasVnrArea2_200
MasVnrArea2_250
MasVnrArea2_300
MasVnrArea2_350
MasVnrArea2_400
MasVnrArea2_other
MSSubClas

In [7]:
#
# DO COMMON ACTIONS
#
## flatten categoricals and drop original features
encodeCols = train.select_dtypes(include=['category'])
print encodeCols
#for col in colsToEncode:
#    train = pd.concat([train,pd.get_dummies(train[col])])
    
#train = pd.get_dummies(train)
#test = pd.get_dummies(test)

## store the SalePrice as log of original
#train["SalePrice"] = train["SalePrice"].apply(np.log)

print train.shape
for col in train.columns:
    print col

     MSSubClass MSZoning Street Alley LotShape LandContour Utilities  \
0            60       RL   Pave   NaN      Reg         Lvl    AllPub   
1            20       RL   Pave   NaN      Reg         Lvl    AllPub   
2            60       RL   Pave   NaN      IR1         Lvl    AllPub   
3            70       RL   Pave   NaN      IR1         Lvl    AllPub   
4            60       RL   Pave   NaN      IR1         Lvl    AllPub   
5            50       RL   Pave   NaN      IR1         Lvl    AllPub   
6            20       RL   Pave   NaN      Reg         Lvl    AllPub   
7            60       RL   Pave   NaN      IR1         Lvl    AllPub   
8            50       RM   Pave   NaN      Reg         Lvl    AllPub   
9           190       RL   Pave   NaN      Reg         Lvl    AllPub   
10           20       RL   Pave   NaN      Reg         Lvl    AllPub   
11           60       RL   Pave   NaN      IR1         Lvl    AllPub   
12           20       RL   Pave   NaN      IR2         Lvl    Al

In [9]:
train.to_csv('data/train_fe.csv',index=False)
test.to_csv('data/test_fe.csv',index=False)
with open('data/features_fe.txt', "w") as feature_file:
    feature_file.write(np.array2string(train.columns, separator=","))

In [23]:
##
## convert categoricals to integers
##
after_train = train.copy()

#after_dev = after_train[~mask].copy()
#after_train = after_train[mask].copy()

train_labels = after_train["SalePrice"].copy()
after_train = after_train.drop( ["SalePrice"], axis=1 )
after_train = after_train.drop( ["Id"], axis=1 )

#dev_labels = after_dev["SalePrice"].copy()
#after_dev = after_dev.drop( ["SalePrice"], axis=1 ) 
#after_dev = after_dev.drop( ["Id"], axis=1 ) 

#print after_train.dtypes
#print after_train.select_dtypes(include=['category']).columns
#print after_train.shape
test = pd.get_dummies(after_train, columns = after_train.select_dtypes(include=['category']).columns)
test = test.drop( ["LotFrontage", "GarageYrBlt"], axis=1 )
total = test.isnull().sum().sort_values(ascending=False)
print total.head(10)

reg = LinearRegression()
reg.fit(test, train_labels)
#print("R^2 value: " + str(reg.score(after_dev, dev_labels)))

SaleCondition_Partial    0
MSSubClass_160           0
LotShape_IR2             0
LotShape_IR3             0
LotShape_Reg             0
LandContour_Bnk          0
LandContour_HLS          0
LandContour_Low          0
LandContour_Lvl          0
Utilities_AllPub         0
dtype: int64


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)